In [2]:
import fiftyone as fo

from dataset import get_transforms, FiftyOneTorchDataset, collate_fn
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import engine

Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.


In [3]:
batch_size = 5
num_epochs = 30
TRAIN_TEST_SPLIT = .9

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
TRAINED_MODEL_FILENAME = "halo_FRCNN_model.pth"

# re load the dataset
if len(fo.list_datasets()) > 0:
    dataset = fo.load_dataset("halo-dataset")
else:
    name = "halo-dataset"
    data_path = "C:/Users/blain\Documents\Git\AutoAim/halo_data/"
    labels_path = "C:/Users/blain\Documents\Git\AutoAim/halo_data/labels/"

    classes = ["enemy"]

    # Import dataset by explicitly providing paths to the source media and labels
    dataset = fo.Dataset.from_dir(
        dataset_type=fo.types.YOLOv4Dataset,
        data_path=data_path,
        labels_path=labels_path,
        classes=classes,
        name=name,
    )

dataset.compute_metadata()
session = fo.launch_app(dataset)


In [4]:
train_transforms = get_transforms(train=True)
test_transforms = get_transforms(train=False)

# split the dataset in train and test set
train_view = dataset.take((len(dataset) * TRAIN_TEST_SPLIT), seed=51)
test_view = dataset.exclude([s.id for s in train_view])

print(f'Traning on {len(train_view)} samples')
print(f'Testing on {len(test_view)} samples')

# use our dataset and defined transformations
train_dataset = FiftyOneTorchDataset(train_view, train_transforms,)
evaluation_dataset = FiftyOneTorchDataset(test_view, test_transforms)

#session.view = train_view

Traning on 45 samples
Testing on 5 samples


In [5]:
train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
    collate_fn=collate_fn,
    drop_last=False
)

valid_data_loader = torch.utils.data.DataLoader(
    evaluation_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
    collate_fn=collate_fn,
    drop_last=True
)

In [6]:
img, target = evaluation_dataset[0]
# put the model in evaluation mode


torch.Size([3, 640, 640])


In [13]:
# load a pre-trained pre-trained FRCNN from torch
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one using our one enemy class.  Use 2 here because we need a background class
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)

# send the model to the training device
model.to(DEVICE)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
LEARNING_RATE = 3e-4
WEIGHT_DECAY = 5e-4
optimizer = torch.optim.Adam(params, lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

scaler = torch.cuda.amp.GradScaler()

In [14]:
best_loss = 999

train_losses = []
eval_losses = []

for epoch in range(num_epochs):
    # training for one epoch
    training_loss = engine.train_one_epoch(model, optimizer, train_data_loader, DEVICE, epoch,scaler=scaler, print_freq=1)


    print('training loss: ', training_loss.meters['loss'].avg)

    train_losses.append(training_loss)

    # evaluate on the test dataset
    engine.evaluate(model, valid_data_loader, device=DEVICE)

    if training_loss.meters['loss'].avg <= best_loss:
        best_loss = training_loss.meters['loss'].avg
        torch.save({'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict()},
                   f'FRCNN_{TRAINED_MODEL_FILENAME}')
        print(f'\nSaving epoch {epoch} - Training Loss {best_loss}\n')


Epoch: [0]  [0/9]  eta: 0:00:33  lr: 0.000038  loss: 0.8719 (0.8719)  loss_classifier: 0.7709 (0.7709)  loss_box_reg: 0.0683 (0.0683)  loss_objectness: 0.0274 (0.0274)  loss_rpn_box_reg: 0.0053 (0.0053)  time: 3.7722  data: 0.1010  max mem: 3465
Epoch: [0]  [1/9]  eta: 0:00:17  lr: 0.000075  loss: 0.8529 (0.8624)  loss_classifier: 0.7605 (0.7657)  loss_box_reg: 0.0557 (0.0620)  loss_objectness: 0.0272 (0.0273)  loss_rpn_box_reg: 0.0053 (0.0074)  time: 2.1611  data: 0.0910  max mem: 3946
Epoch: [0]  [2/9]  eta: 0:00:11  lr: 0.000113  loss: 0.8529 (0.7253)  loss_classifier: 0.7605 (0.6185)  loss_box_reg: 0.0683 (0.0777)  loss_objectness: 0.0272 (0.0220)  loss_rpn_box_reg: 0.0063 (0.0071)  time: 1.6206  data: 0.0847  max mem: 3946
Epoch: [0]  [3/9]  eta: 0:00:08  lr: 0.000150  loss: 0.4512 (0.6015)  loss_classifier: 0.3243 (0.4856)  loss_box_reg: 0.0683 (0.0799)  loss_objectness: 0.0272 (0.0273)  loss_rpn_box_reg: 0.0063 (0.0088)  time: 1.3497  data: 0.0810  max mem: 3946
Epoch: [0]  [4/9

In [15]:


def apply_nms(orig_prediction, iou_thresh=0.3):

    # torchvision returns the indices of the bboxes to keep
    keep = torchvision.ops.nms(orig_prediction['boxes'], orig_prediction['scores'], iou_thresh)

    final_prediction = orig_prediction
    final_prediction['boxes'] = final_prediction['boxes'][keep]
    final_prediction['scores'] = final_prediction['scores'][keep]
    final_prediction['labels'] = final_prediction['labels'][keep]

    return final_prediction

# function to convert a torchtensor back to PIL image
def torch_to_pil(img):
    return torchvision.transforms.ToPILImage()(img).convert('RGB')

import tkinter
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')
import matplotlib.patches as patches

def plot_img_bbox(img, target):
    # plot the image and bboxes
    # Bounding boxes are defined as follows: x-min y-min width height
    fig, a = plt.subplots(1,1)
    fig.set_size_inches(5,5)
    a.imshow(img)
    for box in (target['boxes']):
        x, y, width, height  = box[0], box[1], box[2]-box[0], box[3]-box[1]
        rect = patches.Rectangle((x, y),
                                 width, height,
                                 linewidth = 2,
                                 edgecolor = 'r',
                                 facecolor = 'none')

        # Draw the bounding box on top of the image
        a.add_patch(rect)
    plt.show()


In [1]:
# pick one image from the test set
img, target = evaluation_dataset[0]
# put the model in evaluation mode
model.eval()

with torch.no_grad():
    prediction = model([img.to(DEVICE)])[0]

cpu_pred = {}
for key, value in prediction.items():
    cpu_pred[key] = value.cpu()

print('predicted #boxes: ', len(prediction['labels']))
print('real #boxes: ', len(target['labels']))

print('EXPECTED OUTPUT')
plot_img_bbox(torch_to_pil(img), target)

print('Predicted OUTPUT')
plot_img_bbox(torch_to_pil(img), cpu_pred)

nms_prediction = apply_nms(cpu_pred, iou_thresh=0.2)
print('NMS APPLIED MODEL OUTPUT')
plot_img_bbox(torch_to_pil(img), nms_prediction)

NameError: name 'evaluation_dataset' is not defined